In [13]:
import pandas as pd
import os
from dotenv import load_dotenv
from openai import OpenAI
import chromadb

In [3]:
# carregando
load_dotenv()
try:
    df = pd.read_csv(
    os.path.join(os.getenv("DATA_PATH"), "Restaurant_Reviews.tsv"),
    sep="\t"
)
    #print(df.columns)
    df_reviews = df[['Review']]
    #print(df_reviews.head())
    docs = df_reviews['Review'].tolist()
except FileNotFoundError:
    print("Arquivo não encontrado")

ids = [f"review_{i}" for i in range(len(docs))]

print(f"Carregados {len(docs)} reviews.")
print(f"Exemplo: {docs[0]}")

Carregados 1000 reviews.
Exemplo: Wow... Loved this place.


In [4]:
# embbedando
client = chromadb.Client()
collection = client.create_collection(name="food_reviews")
collection.add(documents=docs, ids=ids)

/home/eduarda/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:24<00:00, 3.46MiB/s]


In [36]:
# teste
question = "What do my customers think about my customer service?"
results = collection.query(
    query_texts=[question],
    n_results=50
)
for i, review in enumerate(results['documents'][0]):
    print(f"{i+1}. {review}")

print("\n")

1. Everyone is very attentive, providing excellent customer service.
2. Service was good and the company was better!
3. As for the service, I thought it was good.
4. Talk about great customer service of course we will be back.
5. The service was great, even the manager came and helped with our table.
6. We've have gotten a much better service from the pizza place next door than the services we received from this restaurant.
7. As for the service: I'm a fan, because it's quick and you're being served by some nice folks.
8. Service was exceptional and food was a good as all the reviews.
9. Service was fantastic.
10. Service was fine and the waitress was friendly.
11. Service is friendly and inviting.
12. Service is quick and friendly.
13. Based on the sub-par service I received and no effort to show their gratitude for my business I won't be going back.
14. The price is reasonable and the service is great.
15. Good service, very clean, and inexpensive, to boot!
16. Bad day or not, I have

In [42]:
def construir_prompt_agente(pergunta_usuario, reviews_recuperados):
    contexto = "\n".join([f"- {r}" for r in reviews_recuperados])
    
    prompt = f"""
    You are an experienced iFood Customer Success Manager.

    YOUR TASK:
    Analyze the following customer reviews retrieved from the database: {contexto}

    The restaurant owner’s question is: "{pergunta_usuario}"

    Do not analyze review by review. Analyze the DATASET AS A WHOLE and generate an executive report.

    YOUR INSTRUCTIONS:
    1. What is the predominant sentiment? (e.g., 80% Positive / 20% Negative) — make an estimate based on your reading.
    2. Severity check:
        * If the reviews indicate serious issues (food poisoning, extreme rudeness, spoiled food), write: **"RISK ALERT: [Reason]"** and generate a draft apology offering a 20% discount.
        * Otherwise, provide improvement suggestions only.
        * State that this is your recommendation to the restaurant owner.
    3. Omit your chain-of-thought from the final user-facing output.
    4. Remember: you are an assistant. Your goal is to make the user’s experience as easy as possible by analyzing the reviews, so omit unnecessary details.
    5. You do not want to disappoint the restaurant owner, so avoid using the same wording customers used in their reviews.
    6. Never fabricate data. For every conclusion, ensure there is real evidence to support it.
    7. Do not write sentences between asteristcs in the output.
    """
    return prompt

reviews_encontrados = results['documents'][0] 
pergunta = "What do my customers think about my customer service?"

prompt_final = construir_prompt_agente(pergunta, reviews_encontrados)

print(prompt_final)


    You are an experienced iFood Customer Success Manager.

    YOUR TASK:
    Analyze the following customer reviews retrieved from the database: - Everyone is very attentive, providing excellent customer service.
- Service was good and the company was better!
- As for the service, I thought it was good.
- Talk about great customer service of course we will be back.
- The service was great, even the manager came and helped with our table.
- We've have gotten a much better service from the pizza place next door than the services we received from this restaurant.
- As for the service: I'm a fan, because it's quick and you're being served by some nice folks.
- Service was exceptional and food was a good as all the reviews.
- Service was fantastic.
- Service was fine and the waitress was friendly.
- Service is friendly and inviting.
- Service is quick and friendly.
- Based on the sub-par service I received and no effort to show their gratitude for my business I won't be going back.
- The

In [44]:
# agente
api_key = "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"

client_llm = OpenAI(base_url="https://api.groq.com/openai/v1",api_key=api_key)

def agent(prompt):
    print("Aguarde...")
    try:
        response = client_llm.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": "You are an agent especialized in analyzing restaurants reviews. Use only the context provided to answer any requests."},
                {"role": "user", "content": prompt}
                ],
            temperature=1
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Erro: {e}"

anwser = agent(prompt_final)
print(anwser)


Aguarde...
Executive Report: Customer Service Review Analysis

The predominant sentiment of the customer reviews is approximately 80% Positive and 20% Negative.

After analyzing the dataset, I did not find any serious issues that would warrant a risk alert, such as food poisoning or extreme rudeness. However, some customers did mention poor service, rudeness, and slow service.

My recommendation to the restaurant owner is to focus on improving customer service consistency. Suggested improvements include providing ongoing staff training to ensure all customers receive friendly and attentive service, implementing a system to promptly address customer complaints, and recognizing and rewarding staff members who provide exceptional service.

Overall, the restaurant is on the right track, with many customers praising the service, atmosphere, and food. By addressing the areas of improvement, the restaurant can further enhance its customer satisfaction and loyalty.
